In [1]:
import numpy as np
import copy

Interesting details on weight initialization https://pouannes.github.io/blog/initialization/

Use Kaiming method 

"The Kaiming paper accordingly suggests to initialize the weights of layer l with a zero-mean Gaussian distribution with a standard deviation of sqrt(s/Nl) , and null biases."

Nl is the number of neurons in layer l

In [3]:
def relu(number):
    """
    Returns the ReLU of a number
    ReLU function is 0 for all negative inputs, and f(x) = x for all x >= 0
    """
    return max(0, number)

In [306]:
class Neuron:
    """
    Neuron class for neural network
    Each neuron is a node in the network
    Can be organized into layers with Layer class
    Each Neuron has a list of Neurons coming in (the upstream neurons in the network)
    and a list of Neurons going out (the downstream neurons in the network)
    The weights correspond to the list of input Neurons
    ex. a Neuron with 5 upstream neurons will have 5 weights, one for each input
    Bias determines a Neurons tendency to be off/on
    """
    def __init__(self, activation=0, inNeurons=[], outNeurons=[], weights=[], bias=0):
        """
        Constructor for Neuron class
        """
        self.inNeurons = inNeurons
        self.outNeurons = outNeurons
        self.activation = activation
        self.weights = weights
        self.bias = bias
        
    def addIn(self, n, weight):
        """
        Adds input neuron with provided weight
        Used in Layer class to connect Layers
        """
        self.inNeurons.append(n)
        self.weights.append(weight)
        
    def addOut(self, neuron):
        """
        Adds output neuron
        Used in Layer class to connect Layers
        """
        self.outNeurons.append(neuron)
    
    def getActivation(self):
        """
        Returns activation of Neuron
        """
        return self.activation
    
    def setActivation(self, a):
        """
        Sets activation of Neuron to a
        Returns a
        """
        self.activation = a
        return a
    
    def getBias(self):
        """
        Returns bias of Neuron
        """
        return self.bias
    
    def setBias(self, b):
        """
        Sets bias of Neuron to b
        Returns b
        """
        self.bias = b
        return b


class Layer:
    """
    Layer class for neural network
    Layers are collections of neurons
    Layers can be linked together to form networks
    """
    def __init__(self, neurons=[], generate=True, size=0):
        """
        Constructor for Layer class
        Can input a list of neurons to build layer, or randomly generate
        neurons for layer of desired size
        """
        self.neurons = neurons.copy()
        print("Initializing Layer")
        
        #  Generates the neurons for the layer
        if generate and len(self.neurons) == 0:
            for i in range(size):
                self.neurons.append(Neuron())
        
        print("Created layer of size %d" % len(self.neurons))
        
        #  Initializes attributes
        self.size = size
        self.upLayer = None
        self.downLayer = None
        self.weights = None
        self.weightsSet = False
        
        #  Records activations and biases of neurons in layer
        self.biases = np.array(list(map(Neuron.getBias, self.neurons)))
        self.activations = np.array(list(map(Neuron.getActivation, self.neurons)))
    
    def setActivations(self, actList):
        """
        Sets activations of neurons in layer to actList
        """
        #  Checks to ensure provided activation list is the appropriate size
        if len(actList) == self.size:
            for i in range(self.size):
                self.neurons[i].setActivation(actList[i])
            self.activations = actList
        else:
            raise Exception("Invalid activation list length")
                
    
    def getActivations(self):
        """
        Returns activations of Neurons in Layer
        """
        return self.activations
    
    def getBiases(self):
        """
        Returns biases of Neurons in Layer
        """
        return self.biases
    
    def getSize(self):
        """
        Returns number of Neurons in Layer
        """
        return self.size
    
    def downstreamConnect(self, down, weights=None):
        """
        Connects neuron layer "down" to self 
        Connection is such that "down" is downstream in the neural network
        Unless specified, weight matrix is initialized randomly within Kaiming distribution
        layer1.upstreamConnect(layer2) is equivalent to layer2.downstreamConnect(layer1)
        IMPORTANT NOTE: WEIGHT MATRIX MIGHT NOT PROPERLY FOLLOW KAIMING
        INSTEAD THEY SAMPLE RANDOMLY FROM A GAUSSIAN DISTRIBUTION WITH VARIANCE DETERMINED BY KAIMING
        MAY CAUSE PROBLEMS WITH SMALL LAYERS, POSSIBLY NEEDS FIX LATER
        """
        upLayerNeurons = self.neurons
        downLayerNeurons = down.neurons
        
        if not weights:
            #  Create random weight initialization matrix
            #  Weights are picked randomly from gaussian of mean=0 and variance according to Kaiming
            weightVariance = np.sqrt(2/len(upLayerNeurons))
            weights = np.random.normal(scale=weightVariance, size=(down.getSize(), self.getSize()))
            print("Created %d by %d weight matrix" % weights.shape)
            down.weightsSet = True
            
        for d in range(down.getSize()):
            for u in range(self.getSize()):
                #  Connect all Neurons between the Layers
                #  Set weights of Neurons in downstream Layer
                upLayerNeurons[u].addOut(downLayerNeurons[d])
                downLayerNeurons[d].addIn(upLayerNeurons[u], weights[d, u])
        
        #  Set weight matrix of downstream Layer and mark up/downstream connections
        down.weights = weights  #  This should be a method like updateWeights(), might be useful later
        down.upLayer = self
        self.downLayer = l
        
        return weights
    
    def upstreamConnect(self, up):
        """
        IMPLEMENTATION INCOMPLETE
        Connects neuron layer "up" to self 
        Connection is such that "up" is upstream in the neural network
        layer1.upstreamConnect(layer2) is equivalent to layer2.downstreamConnect(layer1)
        """
    
    def update(self):
        """
        Updates Neuron activations based on weight matrix, biases and activations of upstream Layer
        """
        #  Only update if weights had been set
        if self.weightsSet:
            weightedSum = np.matmul(self.weights, self.upLayer.getActivations())
            weightedSum = np.subtract(weightedSum, self.biases)
            newActivations = np.array(list(map(relu, weightedSum)))
            self.setActivations(newActivations)


In [307]:
class NN:
    """
    Neural Network class
    A NN consists of Neuron Layers, connected in a linear manner
    End Layers act as data input and output
    Middle Layers each have upstream and downstream connections
    and serve as NN's hidden Layers
    """
    def __init__(self, numLayers, layerSizes):
        """
        Constructor for NN class
        numLayers is an integer that determines the number of Layers in the NN
        layerSizes is a list or tuple of length numLayers that determines the
        number of Neurons in each layer, with layerSizes[0] being the size of the input layer
        """
        self.numLayers = numLayers
        self.layerSizes = layerSizes
        print("Creating Neural Network with %d layers" % numLayers)
        self.layers = []
        
        #  Create layers and connect them
        for i in range(numLayers):
            self.layers.append(Layer(size=layerSizes[i]))
            if i > 0:
                #  No downstream connection for the last layer
                self.layers[i-1].downstreamConnect(self.layers[i])
        print("Neural Network Initialized")
        
    def inputData(self, data):
        """
        Sets input layer activations to data if data is the proper size
        Otherwise raises Exception
        """
        if self.layers[0].getSize() == len(data):
            self.layers[0].setActivations(data)
        else:
            raise Exception("Invalid data size")
    
    def showStructure(self):
        """
        Prints out the number of layers in the NN
        Prints out the sizes of each layer
        """
        print("The network is contains %d layers" % numLayers)
        print("The layer sizes are %s" % layerSizes)
        
    def update(self):
        """
        Updates all layers in NN according to their weight matrix and the activations
        of their upstream Layer
        """
        #  Input layer is skipped, as it has no upstream layer
        for layer in self.layers[1:]:
            layer.update()
            
    def showActivations(self):
        """
        Displays activations of each Layer of the NN
        """
        for i in range(10):
            print(self.layers[i].getActivations())
    

In [308]:
layer1 = Layer(size=10)
layer1.setActivations(list(range(10)))
print("layer1 activations: %s" % layer1.activations)
layer2 = Layer(size=5)

w = layer1.downstreamConnect(layer2)
print(layer1.weights)
print(layer2.weights)
print("layer2 activations: %s" % layer2.activations)
layer2.update()
print("layer2 activations: %s" % layer2.activations)

Initializing Layer
Created layer of size 10
layer1 activations: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Initializing Layer
Created layer of size 5
Created 5 by 10 weight matrix
None
[[ 0.41826406 -0.73508648 -0.91252122 -0.55303445 -0.44873905 -0.2083074
   0.78527366  0.93844238  0.78887115  0.92341751]
 [-0.00826183  0.42273283 -0.14165828 -0.08958547 -0.18177151 -0.26046132
   0.31217827 -0.34199467  0.63927707  0.23725629]
 [ 0.33466217  0.30406361 -0.3573735  -0.36564607 -0.70257798 -0.15948072
   0.46747935  0.1833991  -0.37148799 -0.29806492]
 [-0.50984699 -0.53992274  0.05835959  0.06526827 -0.65753727 -0.68765525
  -0.38695866 -0.152884    0.90457353 -0.53596613]
 [-0.28145428  0.10075495  0.08126343 -0.57253182 -0.21441242 -0.06520552
  -0.85161343 -0.32746237 -0.16212719  0.71408536]]
layer2 activations: [0 0 0 0 0]
layer2 activations: [18.84673994  4.56989734  0.          0.          0.        ]


In [309]:
nn = NN(10, [10]*10)
data = [2]*10
nn.inputData(data)
nn.showActivations()
nn.update()
nn.showActivations()

Creating Neural Network with 10 layers
Initializing Layer
Created layer of size 10
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Initializing Layer
Created layer of size 10
Created 10 by 10 weight matrix
Neural Network Initialized
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0